In [18]:
import numpy as np
import os
import pandas as pd
import yaml
np.random.seed(42)
import os
from resum.polynomial_chaos_expansion import PCEMultiFidelityModel
import pymc as pm
import matplotlib.pyplot as plt
import arviz as az


In [ ]:
with open("../binary-black-hole/settings.yaml", "r") as f:
    config_file = yaml.safe_load(f)

version       = config_file["path_settings"]["version"]
path_out_cnp  = config_file["path_settings"]["path_out_cnp"]
path_out_pce = config_file["path_settings"]["path_out_pce"]
file_in=f'{path_out_cnp}/cnp_{version}_output.csv'


In [ ]:

if not os.path.exists(path_out_pce):
   os.makedirs(path_out_pce)

# Set parameter name/x_labels -> needs to be consistent with data input file
x_labels        = config_file["simulation_settings"]["theta_headers"]
y_label_cnp     = 'y_cnp'
y_err_label_cnp = 'y_cnp_err'
y_label_sim     = 'y_raw'

# Set parameter boundaries
xmin = config_file["simulation_settings"]["theta_min"]
xmax = config_file["simulation_settings"]["theta_max"]
x_fixed = config_file["simulation_settings"]["theta_fixed"]
parameters={}
for i,x in enumerate(x_labels):
   parameters[x]=[xmin[i],xmax[i]]


In [ ]:
data=pd.read_csv(file_in)

In [ ]:

LF_cnp_noise=np.mean(data.loc[(data['fidelity']==0.) & (data['iteration']==0)][y_err_label_cnp].to_numpy())
HF_cnp_noise=np.mean(data.loc[(data['fidelity']==1.) & (data['iteration']==0)][y_err_label_cnp].to_numpy())

x_train_hf = data.loc[(data['fidelity']==1.) & (data['iteration']==0)][x_labels].to_numpy()
y_train_hf = data.loc[(data['fidelity']==1.) & (data['iteration']==0)][y_label_sim].to_numpy()

x_train_mf = data.loc[(data['fidelity']==1.) & (data['iteration']==0)][x_labels].to_numpy()
y_train_mf = data.loc[(data['fidelity']==1.) & (data['iteration']==0)][ y_label_cnp].to_numpy()

x_train_lf_sim = data.loc[(data['fidelity']==0.) & (data['iteration']==0)][x_labels].to_numpy()
y_train_lf_sim = data.loc[(data['fidelity']==0.) & (data['iteration']==0)][ y_label_sim].to_numpy()

x_train_lf = data.loc[(data['fidelity']==0.) & (data['iteration']==0)][x_labels].to_numpy()
y_train_lf = data.loc[(data['fidelity']==0.) & (data['iteration']==0)][ y_label_cnp].to_numpy()

In [ ]:
# Initialize the model
trainings_data = {"lf": [x_train_lf,y_train_lf], 
#    "mf": [x_train_mf, y_train_mf], 
#    "hf": [x_train_hf,y_train_hf]
}

priors = {
    "lf": {"sigma_coeffs": 0.5, "sigma": 0.01},
#    "mf": {"sigma_rho": 0.05, "sigma_coeffs_delta": 0.05, "sigma": 0.01},
#    "hf": {"sigma_rho": 0.05, "sigma_coeffs_delta": 0.05, "sigma": 0.01}
}

In [ ]:
def run_model(degree):
    # Initialize the multi-fidelity model
    multi_fidelity_model = None
    multi_fidelity_model = PCEMultiFidelityModel(trainings_data, priors, degree=degree)
    multi_fidelity_model.build_model()
    multi_fidelity_model.sanity_check_of_basis()
    with multi_fidelity_model.model:
        trace = pm.sample(draws=700, tune=100, chains=1, target_accept=0.99, init="adapt_diag",cores=1, return_inferencedata=True, log_likelihood=True, progressbar=True)
    az.plot_trace(trace)
    plt.tight_layout()
    plt.show()
    az.plot_energy(trace)  # reveals E-BFMI problems
    az.plot_forest(trace, var_names=["sigma_lf", "coeffs_lf"])
    multi_fidelity_model.add_log_likelihood_manually(trace)
    return multi_fidelity_model, trace

In [ ]:
model_1, trace_1 = run_model(degree=1)
model_2, trace_2 = run_model(degree=2)
model_3, trace_3 = run_model(degree=3)
model_6, trace_6 = run_model(degree=6)

In [ ]:
comparison = az.compare({"deg1": trace_1, "deg2": trace_2, "deg3": trace_3, "deg6": trace_6}, ic="loo")
print(comparison)
az.plot_compare(comparison)